In [ ]:
## Green space

import ee
from google.colab import drive
ee.Authenticate()
ee.Initialize()


To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=94ngemqzoeAVqX7rlu1kdeJ9jqM2F_E5tfzDAkndU2c&tc=pf1rTul2pz-oF_71sTzIhq1xY4K8Ug_8YbxV6oElj-Q&cc=YtdGshFqI4kIbvCUf7EEt0SV2iTY7nXZjXjOSPbNsRM

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AbUR2VPY8PCDAEeoUiIGA1ZWnHuAXnU0OdIBjA0I6vzmLL8tFZ4xFgXIUgg

Successfully saved authorization token.


In [ ]:
# Define the coordinates (longitude, latitude) and image scale
longitude = -71.117736
latitude = 42.265168
scale = 50  # 100 meters


# Select a satellite imagery dataset
# dataset = ee.ImageCollection('LANDSAT/LC09/C02/T1')
dataset = ee.ImageCollection("USDA/NAIP/DOQQ")
# dataset = ee.ImageCollection('COPERNICUS/S2_SR')  # Sentinel-2 Level-1C

# Define the region of interest (ROI)
roi = ee.Geometry.Point([longitude, latitude])

# Filter the dataset by date, region, and cloud cover
filtered_dataset = dataset.filterDate('2015-04-01', '2015-05-01') \
                          # .filterBounds(roi) \
                          # .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 10)

# Get the median value of the filtered dataset
image = filtered_dataset.median()

# Clip the image to the region of interest
clipped_image = image.clip(roi.buffer(scale))

# Generate the image URL
url = clipped_image.getThumbURL({
    'region': roi.buffer(scale).bounds().getInfo()['coordinates'],
    'dimensions': [1024, 1024],
    'bands': 'R,G,B',  # RGB bands
    'format': 'png',
    'min': 0.0,
    'max': 255.0
})

print("Image URL:", url)


ee.Image({
  "functionInvocationValue": {
    "functionName": "Collection.first",
    "arguments": {
      "collection": {
        "functionInvocationValue": {
          "functionName": "Collection.filter",
          "arguments": {
            "collection": {
              "functionInvocationValue": {
                "functionName": "ImageCollection.load",
                "arguments": {
                  "id": {
                    "constantValue": "USDA/NAIP/DOQQ"
                  }
                }
              }
            },
            "filter": {
              "functionInvocationValue": {
                "functionName": "Filter.dateRangeContains",
                "arguments": {
                  "leftValue": {
                    "functionInvocationValue": {
                      "functionName": "DateRange",
                      "arguments": {
                        "end": {
                          "constantValue": "2015-05-01"
                        },
                 

In [ ]:
task = ee.batch.Export.image.toDrive(image=image, **export_params)
task.start()
print(task.status())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
## Google map
# Latitude and longitude coordinates
latitude = 42.265168
longitude = -71.117736


import requests
from PIL import Image
from io import BytesIO

# Replace with your own Google Maps API key
API_KEY = 'API_KEY'

# Latitude and longitude coordinates
latitude = 42.265168
longitude = -71.117736

# # Calculate the zoom level for a 100-meter range
# zoom = 18  # You might need to adjust this value depending on the location

# # Construct the Google Maps Static API URL
# url = f"https://maps.googleapis.com/maps/api/staticmap?center={latitude},{longitude}&zoom={zoom}&size=256x256&maptype=satellite&key={API_KEY}"

# # Download the image
# response = requests.get(url)
# img = Image.open(BytesIO(response.content))

# # Save the image to a file
# img.save("satellite_image.png")


location = f"{latitude},{longitude}"


base_url = "https://maps.googleapis.com/maps/api/staticmap"
params = {
    "center": location,
    "zoom": 19,
    "size": "512x512",
    "maptype": "satellite",
    # "sensor": "false",
    "scale": 2,  # 1 pixel = 0.5 meters, so 100 meters = 200 pixels
    'key': API_KEY
}

import requests

response = requests.get(base_url, params=params)

with open("map.png", "wb") as f:
    f.write(response.content)





In [ ]:
import ee

# Initialize the Earth Engine API
ee.Initialize()

# Define the coordinates (longitude, latitude) and image scale
longitude = 12.4924
latitude = 41.8902
scale = 30

# Select a satellite imagery dataset
dataset = ee.ImageCollection('LANDSAT/LC09/C02/T1')

# Define the region of interest (ROI)
roi = ee.Geometry.Point([longitude, latitude])

# Create a centered square buffer around the ROI
buffer_size = 50  # Half of the side length of the square buffer (in meters)
buffered_roi = roi.buffer(buffer_size).bounds()

# Filter the dataset by date, region, and cloud cover
filtered_dataset = dataset.filterDate('2020-01-01', '2022-12-31') \
                          .filterBounds(buffered_roi) \
                          .filterMetadata('CLOUD_COVER', 'less_than', 10)

# Get the median value of the filtered dataset
image = filtered_dataset.median()

# Clip the image to the buffered region of interest
clipped_image = image.clip(buffered_roi)

# Generate the image URL
url = clipped_image.getThumbURL({
    'region': buffered_roi.getInfo()['coordinates'],
    'scale': scale,
    'dimensions': [256, 256],
    'bands': 'B4,B3,B2',  # RGB bands
    'format': 'png',
    'min': 0,
    'max': 3000
})

print("Image URL:", url)
